In [1]:
import pandas as pd

In [2]:
df_train = pd.read_csv("final_label_train_fix.csv")
df_test = pd.read_csv("final_label_test.csv")

In [3]:
# len(df_T6), len(df_T9), len(df_training), len(df_day_10_05), len(df_day_10_19), len(df_day_12_02), len(df_day_12_09), len(df_day_12_10), len(df_day_12_12)

In [4]:
# all_sentences = list(set(df_full['text'].tolist()))
# len(all_sentences)

In [5]:
# import re
# from tqdm import tqdm
# chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\']'

# with open('text.txt', 'w') as f:
#     for sentence in tqdm(all_sentences):
#         f.write(sentence)
#         f.write('\n')

In [6]:
from transformers import Wav2Vec2Processor, Wav2Vec2ProcessorWithLM

# Load processor
processor = Wav2Vec2Processor.from_pretrained("custom_model")

# Get the vocabulary dictionary
vocab_dict = processor.tokenizer.get_vocab()

# Remove <s> and </s> tokens if they exist
# vocab_dict = {k: v for k, v in vocab_dict.items() if k not in tokens_to_remove}

# Sort the vocabulary dictionary by the values (IDs)
sorted_vocab_dict = {k.lower(): v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1])}

# Output the sorted vocabulary dictionary
print(sorted_vocab_dict)

/home/jovyan/ai-core/miniconda3/envs/contentvec/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


{'ẻ': 0, '6': 1, 'ụ': 2, 'í': 3, '3': 4, 'ỹ': 5, 'ý': 6, 'ẩ': 7, 'ở': 8, 'ề': 9, 'õ': 10, '7': 11, 'ê': 12, 'ứ': 13, 'ỏ': 14, 'v': 15, 'ỷ': 16, 'a': 17, 'l': 18, 'ự': 19, 'q': 20, 'ờ': 21, 'j': 22, 'ố': 23, 'à': 24, 'ỗ': 25, 'n': 26, 'é': 27, 'ủ': 28, 'у': 29, 'ô': 30, 'u': 31, 'y': 32, 'ằ': 33, '4': 34, 'w': 35, 'b': 36, 'ệ': 37, 'ễ': 38, 's': 39, 'ì': 40, 'ầ': 41, 'ỵ': 42, '8': 43, 'd': 44, 'ể': 45, '|': 46, 'r': 47, 'ũ': 48, 'c': 49, 'ạ': 50, '9': 51, 'ế': 52, 'ù': 53, 'ỡ': 54, '2': 55, 't': 56, 'i': 57, 'g': 58, '́': 59, 'ử': 60, '̀': 61, 'á': 62, '0': 63, 'ậ': 64, 'e': 65, 'ộ': 66, 'm': 67, 'ẳ': 68, 'ợ': 69, 'ĩ': 70, 'h': 71, 'â': 72, 'ú': 73, 'ọ': 74, 'ồ': 75, 'ặ': 76, 'f': 77, 'ữ': 78, 'ắ': 79, 'ỳ': 80, 'x': 81, 'ó': 82, 'ã': 83, 'ổ': 84, 'ị': 85, '̣': 86, 'z': 87, 'ả': 88, 'đ': 89, 'è': 90, 'ừ': 91, 'ò': 92, 'ẵ': 93, '1': 94, 'ơ': 95, 'k': 96, 'ẫ': 97, 'p': 98, 'ấ': 99, 'ẽ': 100, 'ỉ': 101, 'ớ': 102, 'ẹ': 103, 'ă': 104, 'o': 105, 'ư': 106, '5': 107, '<unk>': 108, '<pad>': 109, '

In [7]:
# from transformers import Wav2Vec2CTCTokenizer

# # Save the modified vocabulary to a temporary file
# vocab_file = "updated_vocab.json"
# with open(vocab_file, "w") as f:
#     import json
#     json.dump(vocab_dict, f)

# # Reload tokenizer with updated vocab
# tokenizer = Wav2Vec2CTCTokenizer(vocab_file)

# # Replace the tokenizer in the processor
# processor.tokenizer = tokenizer

In [8]:
from pyctcdecode import build_ctcdecoder

decoder = build_ctcdecoder(
    labels=list(sorted_vocab_dict.keys()),
    kenlm_model_path="5gram_correct.arpa",
    alpha=1
)

Loading the LM will be faster if you build a binary file.
Reading /home/jovyan/ai-core/speech_to_text/nghia_semi_supervised/icra/5gram_correct.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Found entries of length > 1 in alphabet. This is unusual unless style is BPE, but the alphabet was not recognized as BPE type. Is this correct?
Unigrams and labels don't seem to agree.


In [9]:
import kenlm
model = kenlm.Model('5gram_correct.arpa')
print(model.score('chị ơi em đang có đơn hàng gửi từ giao hàng tiết kiệm', bos = True, eos = True))

-16.322723388671875


Loading the LM will be faster if you build a binary file.
Reading /home/jovyan/ai-core/speech_to_text/nghia_semi_supervised/icra/5gram_correct.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************


In [11]:
processor_with_lm = Wav2Vec2ProcessorWithLM(
    feature_extractor=processor.feature_extractor,
    tokenizer=processor.tokenizer,
    decoder=decoder
)

In [12]:
# processor_with_lm.save_pretrained("GHTK_wav2vec2")

In [13]:
class CFG:
    my_model_name = 'custom_model'
    processor_name = 'GHTK_wav2vec2'
    processor_without_LM = 'custom_model'
    
from transformers import Wav2Vec2ProcessorWithLM,pipeline

# processor = Wav2Vec2ProcessorWithLM.from_pretrained(CFG.processor_name)


asr_w_LM = pipeline("automatic-speech-recognition", model=CFG.my_model_name ,feature_extractor=processor_with_lm.feature_extractor, tokenizer=processor_with_lm.tokenizer,decoder=processor_with_lm.decoder)
asr_wo_LM = pipeline("automatic-speech-recognition", model = CFG.my_model_name, feature_extractor=processor.feature_extractor, tokenizer=processor.tokenizer)

/home/jovyan/ai-core/miniconda3/envs/contentvec/lib/python3.7/site-packages/transformers/configuration_utils.py:381: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


In [14]:
import librosa
def infer(audio_path):
    speech, sr = librosa.load(audio_path, sr=processor_with_lm.feature_extractor.sampling_rate)
    my_LM_prediction = asr_w_LM(
                speech
            )

    return my_LM_prediction["text"]

def infer_wo_LM(audio_path):
    speech, sr = librosa.load(audio_path, sr=processor.feature_extractor.sampling_rate)
    my_LM_prediction = asr_wo_LM(
                speech
            )

    return my_LM_prediction['text']

In [15]:
import pandas as pd

def fix_label(path_audio, path_file_csv, df, new_label):
    """
    Sửa nhãn (label) của file audio trong file CSV.

    Args:
        path_audio (str): Đường dẫn đến file audio cần sửa nhãn.
        path_file_csv (str): Đường dẫn đến file CSV chứa dữ liệu.
        new_label (str): Nhãn mới cần gán cho file audio.
    """

    # Kiểm tra file audio có tồn tại trong CSV hay không
    if path_audio not in df['audio'].values:
        print(f"File audio '{path_audio}' không tồn tại trong file CSV.")
        return

    # Sửa nhãn cho file audio
    df.loc[df['audio'] == path_audio, 'labeled_text'] = new_label
    print(f"Đã sửa nhãn của '{path_audio}' thành: '{new_label}'")

    # Ghi lại file CSV
    df.to_csv(path_file_csv, index=False)
    print(f"Đã lưu file CSV với nhãn mới tại: {path_file_csv}")

In [16]:
from IPython.display import Audio
from jiwer import wer

In [62]:
sample = df_train.sample()
path = sample["audio"].values[0]
result_LM = infer(path)
result_wo_LM = infer_wo_LM(path)
print("With LM: ", result_LM)
print("Without LM: ", result_wo_LM)
print("Label: ",sample["labeled_text"].values[0])
print("WER: ",wer(sample["labeled_text"].values[0],result_LM))
Audio(path)

With LM:  al
Without LM:  al
Label:  thế nữa lại
WER:  1.0


In [270]:
ừ em hai ca tư đang ở bên hoàng mai ấy ạ

SyntaxError: invalid syntax (2606571536.py, line 1)

In [317]:
fix_sentence = input("Nhập câu đúng:")
fix_label(path, "final_label_train_fix.csv", df_train, fix_sentence)

Đã sửa nhãn của '/home/jovyan/ai-core/speech_to_text/data/ghtk_T6/voice_split/72369-842471016869-0965958182-20240604103513-1717472100.584099.wav-out_segment_0.wav' thành: 'dạ a lô'
Đã lưu file CSV với nhãn mới tại: final_label_train_fix.csv


In [318]:
df_train = pd.read_csv("final_label_train_fix.csv")

In [1]:
id = sample.index.values[0]
print("With LM: ", result_LM)
print("Without LM: ", result_wo_LM)
print("Label: ",df_train.loc[id,"labeled_text"])
print("WER: ", wer(df_train.loc[id,"labeled_text"],result_LM))

NameError: name 'sample' is not defined

In [320]:
path

'/home/jovyan/ai-core/speech_to_text/data/ghtk_T6/voice_split/72369-842471016869-0965958182-20240604103513-1717472100.584099.wav-out_segment_0.wav'

In [261]:
df_train = df_train[df_train["audio"]!=path]
df_train.to_csv("final_label_train_fix.csv",index=False)

In [262]:
df_train = pd.read_csv("final_label_train_fix.csv")

In [263]:
df_train

,audio,labeled_text
0,/home/jovyan/ai-core/speech_to_text/data/ghtk_...,alo ạ
1,/home/jovyan/ai-core/speech_to_text/data/ghtk_...,ờ xử lý hai bao đúng không
2,/home/jovyan/ai-core/speech_to_text/data/ghtk_...,xử lý sớm đi trước mười hai giờ nha
3,/home/jovyan/ai-core/speech_to_text/data/ghtk_...,ừ cho chị hỏi có phải số điện thoại của hương ...
4,/home/jovyan/ai-core/speech_to_text/data/ghtk_...,ừ em có phải là vợ của bạn linh không nhở
...,...,...
17712,/home/jovyan/ai-core/speech_to_text/data/ghtk_...,à à à
17713,/home/jovyan/ai-core/speech_to_text/data/ghtk_...,alo anh ơi em gọi bên giao hàng ý ạ mình có đơ...
17714,/home/jovyan/ai-core/speech_to_text/data/ghtk_...,thế thì chiều nay có nhận được hàng không để e...
17715,/home/jovyan/ai-core/speech_to_text/data/ghtk_...,à vâng để em báo lại ship giao lại cho mình tr...
